# High-Performance Pandas: eval() and query()

앞서 배운 NumPy의 vectorized/broadcasted operation과 Pandas의 grouping-type operation은 효율적 계산을 위해 구현된 기능이다. 하지만 부가적인 메모리가 사용되는 단점이 남아 있는데 이러한 점을 개선하기 위해  [Numexpr](https://github.com/pydata/numexpr) package를 이용한 ``eval()``과  ``query()`` functions 이 Pandas에서 제공된다. 

## Motivating ``query()`` and ``eval()``: Compound Expressions


In [1]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y

100 loops, best of 3: 2.18 ms per loop


위와 같은 Vectorized opertion은 효율적이다. 
그런데 다음과 같은 식을 구현해 보자.

In [0]:
mask = (x > 0.5) & (y < 0.5)

NumPy는 여러 하위식을 계산하게 되는 셈이다. 

In [0]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

중간 단계마다 부가적인 임시 메모리를 사용하는 셈이므로 ``x``와 ``y``가 매우 큰 경우 문제가 될 수 있다. 
Numexpr는 이러한 문제를 해결하기 위해 원소단위로 계산하도록 설계되어 있다. 
(자세한 내용은  [Numexpr documentation](https://github.com/pydata/numexpr)에 있다.)

In [4]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

Pandas ``eval()``과 ``query()``는 이러한 특성을 이용하여 구현되었다. 

## ``pandas.eval()`` for Efficient Operations

다음예와 같이 string expression을 ``eval()`` function에 전달하면 된다. 

In [0]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

In [6]:
%timeit df1 + df2 + df3 + df4

10 loops, best of 3: 76.1 ms per loop


In [7]:
%timeit pd.eval('df1 + df2 + df3 + df4')

10 loops, best of 3: 50 ms per loop


``eval()`` version이 약  50% 더 빠르고 훨씬 적은 메모리를 사용한다. 

In [8]:
np.allclose(df1 + df2 + df3 + df4,
            pd.eval('df1 + df2 + df3 + df4'))

True

### Operations supported by ``pd.eval()``

다음은 Pandas v0.16 기준으로 ``pd.eval()``가 가능한 연산 예들이다. 

In [0]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))

#### Arithmetic operators


In [10]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

#### Comparison operators


In [11]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

#### Bitwise operators


In [12]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

#### Logical operators


In [13]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

#### Object attributes and indices



In [14]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

#### Other operations
기타 operation들은 매뉴얼을 참조하자. 

## ``DataFrame.eval()`` for Column-Wise Operations

``pd.eval()`` function처럼 ``DataFrame`` object도  ``eval()`` method를 가지고 있다. 
특히 *by name*을 통해 column을 참조할 수 있어서 편리하다. 

In [15]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [16]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

심지어 다음과 같이 축약 가능하다. 

In [17]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

### Assignment in DataFrame.eval()

기존 column 혹은 새로운 column으로의 assignment도 가능하다. 

In [18]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [19]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


In [20]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


### Local variables in DataFrame.eval()

`@` 문자를 사용하여 지역 변수 접근이 가능하다. 
이 기능은 ``pandas.eval()`` 에서는 가능하지 않음에 유의하자. 

In [21]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

## DataFrame.query() Method

비슷한 개념으로 필터링을 하기 위한  ``DataFrame``에  ``query()`` method가 있다.


In [22]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

위와 같이 ``pandas.eval()`` function을 통해 필터링을 구현할 수도 있지만 다음과 같이  ``DataFrame.query()`` method를 통해 보기좋게 구현할 수 있따.

In [23]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

``query()`` method에서도  ``@``문자를 이용하여 지역 변수에 접근가능하다. 

In [24]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

## Performance: When to Use These Functions

반복하지만 이러한 방법을 통해 *computation time* 과 *memory use* 측면에서 보다 효율적이다.